In [ ]:
# Create an environment with devbio-napari installed in it.
mamba create --name neurite-devbio-napari-env python=3.9 devbio-napari -c conda-forge

# Activate the environment
conda activate neurite-devbio-napari-env

# Open Jupyter
jupyter lab

### Set folder directory and inspect image shape 

In [ ]:
import os
from skimage.io import imread

folder_dir =  os.path.join(os.path.dirname(__file__), "../images")

# List all TIFF images
files = [f for f in os.listdir(folder_dir) if f.endswith(".tif")]
print(f"Found {len(files)} images.")

# Pick one image (index 0 for example)
img_path = os.path.join(folder_dir, files[0])
img = imread(img_path)
print(f"Loaded {files[0]} with shape: {img.shape}")

print("\nCheck the shape above.")
print("If your image shape is (H, W, C), channel_axis = -1")
print("If your image shape is (C, H, W), channel_axis = 0")
channel_axis = int(input("Enter the channel axis index: "))
print(f"Using channel_axis = {channel_axis} for this dataset.")


### Save cell and background masks, pixel sizes, background values ####

In [ ]:
import os
import numpy as np
import tifffile
import pandas as pd
import napari
from qtpy.QtWidgets import QPushButton

CHANNEL1 = "channel1"  
CHANNEL2 = "channel2"

# 1. Load images
def load_images(folder_dir):
    files = [f for f in os.listdir(folder_dir) if f.endswith(".tif")]
    ImsFP = [os.path.join(folder_dir, f) for f in files]
    ImNames = files
    Ims = [tifffile.imread(path) for path in ImsFP]
    print(f"Loaded {len(Ims)} images from {folder_dir}")
    return Ims, ImsFP, ImNames

# 2. Extract pixel sizes
def extract_pixel_sizes(ImsFP):
    pixel_data = []
    for path in ImsFP:
        with tifffile.TiffFile(path) as tif:
            tags = tif.pages[0].tags
            x_res = tags["XResolution"].value[0] / tags["XResolution"].value[1] if "XResolution" in tags else None
            y_res = tags["YResolution"].value[0] / tags["YResolution"].value[1] if "YResolution" in tags else None
            x_size = 1/x_res if x_res else None
            y_size = 1/y_res if y_res else None
            pixel_area = x_size*y_size if x_size and y_size else None
            pixel_data.append({"file": os.path.splitext(os.path.basename(path))[0], "x_size": x_size, "y_size": y_size, "pixel_area": pixel_area})
    return pd.DataFrame(pixel_data)

# 3. Check for existing masks
def check_existing_masks(folder_dir, file_name):
    mask_dir = os.path.join(folder_dir, "CellMasks")
    if not os.path.exists(mask_dir):
        return None, None

    base = os.path.splitext(file_name)[0]

    # Cell masks
    cell_masks = sorted([
        os.path.join(mask_dir, f) 
        for f in os.listdir(mask_dir) 
        if f.startswith(base + "_cell") and f.endswith(".tif")
    ])
    # Background mask
    bg_mask_path = os.path.join(mask_dir, f"{base}_background.tif")
    if not os.path.exists(bg_mask_path):
        bg_mask_path = None

    return cell_masks, bg_mask_path

# 4. Measure background from existing mask
def measure_background_from_mask(img, bg_mask_path, file_name, bg_csv):
    mask = tifffile.imread(bg_mask_path).astype(bool)
    MIP = img.max(axis=0)

    # MIP metrics
    c1_vals_MIP = MIP[1][mask]
    c2_vals_MIP = MIP[0][mask]
    bg_c1_MIP     = np.mean(c1_vals_MIP)
    bg_c1_MIP_med = np.median(c1_vals_MIP)
    bg_c2_MIP     = np.mean(c2_vals_MIP)
    bg_c2_MIP_med = np.median(c2_vals_MIP)

    # Full Z-stack metrics
    c1_vals_Z = img[:,1][..., mask].flatten()
    c2_vals_Z = img[:,0][..., mask].flatten()
    bg_c1_Z     = np.mean(c1_vals_Z)
    bg_c1_Z_med = np.median(c1_vals_Z)
    bg_c2_Z     = np.mean(c2_vals_Z)
    bg_c2_Z_med = np.median(c2_vals_Z)

    df = pd.DataFrame([[file_name, bg_c1_MIP, bg_c1_MIP_med, bg_c1_Z, bg_c1_Z_med,
                        bg_c2_MIP, bg_c2_MIP_med, bg_c2_Z, bg_c2_Z_med]],
                      columns=["file",
                               f"Bg_{CHANNEL1}_MIP", f"Bg_{CHANNEL1}_MIP_median",
                               f"Bg_{CHANNEL1}_Zmean", f"Bg_{CHANNEL1}_Zmedian",
                               f"Bg_{CHANNEL2}_MIP", f"Bg_{CHANNEL2}_MIP_median",
                               f"Bg_{CHANNEL2}_Zmean", f"Bg_{CHANNEL2}_Zmedian"])
    df.to_csv(bg_csv, mode='a', header=not os.path.exists(bg_csv), index=False)
    print(f"Saved background values for {file_name}")

# 5. Draw masks + background
def draw_masks_and_background(img, MIP, file_name, mask_dir, bg_csv, channel_axis=0):
    os.makedirs(mask_dir, exist_ok=True)
    viewer = napari.view_image(MIP, name=file_name, channel_axis=channel_axis)

    # Cell masks
    mask_layer = viewer.add_shapes(name="Cell masks")
    def save_masks_on_close():
        masks_stack = mask_layer.to_masks(MIP.shape[1:])
        if masks_stack.shape[0] == 0:
            print("No cell masks drawn, skipping save.")
            return
        file_basename = os.path.splitext(file_name)[0]
        for idx, mask in enumerate(masks_stack, start=1):
            mask_to_save = (mask > 0).astype(np.uint8) * 255
            tifffile.imwrite(os.path.join(mask_dir, f"{file_basename}_cell{idx}.tif"), mask_to_save)
        print(f"Saved {len(masks_stack)} masks to: {mask_dir}")

    button_masks = QPushButton("Save cell masks")
    button_masks.clicked.connect(save_masks_on_close)
    viewer.window.add_dock_widget(button_masks)

    # Background
    bg_layer = viewer.add_shapes(name="Background")
    def save_background():
        mask = bg_layer.to_masks(MIP.shape[1:]).max(axis=0)
        if mask.sum() < 1:
            print("No background region drawn!")
            return
        mask_path = os.path.join(mask_dir, f"{os.path.splitext(file_name)[0]}_background.tif")
        tifffile.imwrite(mask_path, (mask > 0).astype(np.uint8)*255)
        print(f"Saved background region for {file_name}")

        # MIP metrics
        mask_bool = mask.astype(bool)
        c1_vals_MIP = MIP[1][mask_bool]
        c2_vals_MIP = MIP[0][mask_bool]
        bg_c1_MIP     = np.mean(c1_vals_MIP)
        bg_c1_MIP_med = np.median(c1_vals_MIP)
        bg_c2_MIP     = np.mean(c2_vals_MIP)
        bg_c2_MIP_med = np.median(c2_vals_MIP)

        # Full Z-stack metrics
        c1_vals_Z = img[:,1][..., mask_bool].flatten()
        c2_vals_Z = img[:,0][..., mask_bool].flatten()
        bg_c1_Z     = np.mean(c1_vals_Z)
        bg_c1_Z_med = np.median(c1_vals_Z)
        bg_c2_Z     = np.mean(c2_vals_Z)
        bg_c2_Z_med = np.median(c2_vals_Z)

        # Save to CSV
        df = pd.DataFrame([[file_name, bg_c1_MIP, bg_c1_MIP_med, bg_c1_Z, bg_c1_Z_med,
                            bg_c2_MIP, bg_c2_MIP_med, bg_c2_Z, bg_c2_Z_med]],
                          columns=["file",
                                   f"Bg_{CHANNEL1}_MIP", f"Bg_{CHANNEL1}_MIP_median",
                                   f"Bg_{CHANNEL1}_Zmean", f"Bg_{CHANNEL1}_Zmedian",
                                   f"Bg_{CHANNEL2}_MIP", f"Bg_{CHANNEL2}_MIP_median",
                                   f"Bg_{CHANNEL2}_Zmean", f"Bg_{CHANNEL2}_Zmedian"])
        df.to_csv(bg_csv, mode='a', header=not os.path.exists(bg_csv), index=False)
        print(f"Saved background values for {file_name}")

    button_bg = QPushButton("Save background region")
    button_bg.clicked.connect(save_background)
    viewer.window.add_dock_widget(button_bg)

    viewer.show(block=True)

# 6. Run pipeline
def run_pipeline(folder_dir, channel_axis=0):
    Ims, ImsFP, ImNames = load_images(folder_dir)
    
    pixel_csv = os.path.join(folder_dir, "pixel_sizes.csv")
    bg_csv    = os.path.join(folder_dir, "background_values.csv")
    
    # Remove existing CSVs so we start fresh
    if os.path.exists(pixel_csv):
        os.remove(pixel_csv)
    if os.path.exists(bg_csv):
        os.remove(bg_csv)

    # Save pixel sizes
    pixel_df = extract_pixel_sizes(ImsFP)
    pixel_df.to_csv(pixel_csv, index=False)

    for img, name in zip(Ims, ImNames):
        base = os.path.splitext(name)[0]
        mask_dir = os.path.join(folder_dir, "CellMasks")
        print(f"\n=== Processing {base} ===")

        # Check existing masks
        cell_masks, bg_mask_path = check_existing_masks(folder_dir, base)

        if bg_mask_path is not None:
            measure_background_from_mask(img, bg_mask_path, base, bg_csv)
            print(f"Measured background from existing mask for {base}")
        else:
            MIP = img.max(axis=0)
            draw_masks_and_background(img, MIP, base, mask_dir, bg_csv, channel_axis=channel_axis)
        print(f"Finished processing {base}")

if __name__ == "__main__":
    run_pipeline(folder_dir, channel_axis=0)

print(f"\nFinished processing for {folder_dir}")

### Open each cell, save a masked cell within the bounding box, also save the cell area ####

In [ ]:
import os
import numpy as np
import tifffile
import pandas as pd

# 1. Extract cropped 4D cell stack + mask area + bbox area
def extract_cell_stack(img_4d, mask_path):
    mask = tifffile.imread(mask_path) > 0

    rows, cols = np.nonzero(mask)
    if len(rows) == 0:
        return None, None, None, None

    min_r, max_r = np.min(rows), np.max(rows)
    min_c, max_c = np.min(cols), np.max(cols)

    cropped = img_4d[:, :, min_r:max_r+1, min_c:max_c+1]

    mask_area = np.sum(mask)
    bbox_area = (max_r - min_r + 1) * (max_c - min_c + 1)

    return cropped, mask_area, bbox_area, (min_r, max_r, min_c, max_c)

# 2. Save cropped cell stack to TIFF
def save_cropped_cell(cropped_stack, output_dir, base, cell_index):
    os.makedirs(output_dir, exist_ok=True)

    save_path = os.path.join(output_dir, f"{base}_cell{cell_index}.tif")
    tifffile.imwrite(save_path, cropped_stack.astype(cropped_stack.dtype), imagej=True, metadata={'axes': 'ZCYX'})

    return save_path

# 3. Run pipeline
def process_cells(folder_dir):
    mask_dir = os.path.join(folder_dir, "CellMasks")
    output_dir = os.path.join(folder_dir, "Cells")
    os.makedirs(output_dir, exist_ok=True)

    csv_path = os.path.join(folder_dir, "cell_measurements.csv")
    cells_list = []
    cell_names = []
    entries = []

    # Loop over original images
    for fname in os.listdir(folder_dir):
        if not fname.endswith(".tif"):
            continue

        base = os.path.splitext(fname)[0]
        print(f"\n=== Processing {base} ===")

        img_path = os.path.join(folder_dir, fname)
        img_4d = tifffile.imread(img_path)

        # Collect masks for this image
        file_basename = os.path.splitext(os.path.basename(base))[0]
        mask_files = sorted(
            [f for f in os.listdir(mask_dir) if f.startswith(file_basename + "_cell")]
        )

        if len(mask_files) == 0:
            print("No masks found for this image.")
            continue

        for idx, mask_file in enumerate(mask_files, start=1):
            mask_path = os.path.join(mask_dir, mask_file)

            # --- 1. Extract cropped stack ---
            cropped, mask_area, bbox_area, bbox_coords = extract_cell_stack(img_4d, mask_path)

            if cropped is None:
                print(f"Empty mask for {mask_file}")
                continue

            # --- 2. Save cropped TIFF ---
            cell_tif_path = save_cropped_cell(cropped, output_dir, file_basename, idx)


            # --- Collect results ---
            entries.append([
                base,
                f"{file_basename}_cell{idx}.tif",
                mask_area,
                bbox_area,
                cell_tif_path,
                mask_path
            ])

    # Save results to CSV
    df = pd.DataFrame(
        entries,
        columns=["file", "cell_image", "mask_area", "bbox_area", "cell_tif_path", "mask_path"]
    )
    df.to_csv(csv_path, index=False)

    print(f"\nSaved measurements to: {csv_path}")

def run_cell_pipeline(folder_dir):
    """
    Run AFTER mask drawing pipeline.
    Extract each cell, save TIFF, compute areas
    """
    process_cells(folder_dir)

run_cell_pipeline(folder_dir)

### Manually open each cell in imageJ and make a csv file with the filename and focal plane ####

### Combine all csv files into one "all_cell_details" csv file

In [19]:
import pandas as pd
import os

pixel_csv = os.path.join(folder_dir, "pixel_sizes.csv")
bg_csv = os.path.join(folder_dir, "background_values.csv")
measure_csv = os.path.join(folder_dir, "cell_measurements.csv")
manual_csv = os.path.join(folder_dir, "cell_ManualDetails.csv")

# Load the CSV files
pixel_df = pd.read_csv(pixel_csv)
bg_df = pd.read_csv(bg_csv)
measure_df = pd.read_csv(measure_csv)
manual_df = pd.read_csv(manual_csv, sep = ";")
manual_df = manual_df.loc[:, ~manual_df.columns.str.contains("^Unnamed")]

# Merge on the "file" column
merged_df1 = pd.merge(pixel_df, bg_df, on="file", how="left")

# Merge on the "cell_image" column
merged_df2 = pd.merge(measure_df, manual_df, on="cell_image", how="left")
# Remove duplicated "file" column
if "file_x" in merged_df2.columns and "file_y" in merged_df2.columns:
    merged_df2["file"] = merged_df2["file_x"]
    merged_df2 = merged_df2.drop(columns=["file_x", "file_y"])

# Merge everything together
merged_df = pd.merge(merged_df2, merged_df1, on="file", how="left")      

# Save
output_path = os.path.join(folder_dir, "all_cell_details.csv")
merged_df.to_csv(output_path, index=False)

del pixel_df, bg_df, measure_df, manual_df, merged_df1, merged_df2, merged_df

### Open each image and extract only the focal adhesion plane

In [12]:
import pandas as pd
import os
import tifffile

# Load CSV
df_cell_details = pd.read_csv(os.path.join(folder_dir, "all_cell_details.csv"))
df_cell_details["cell_image"] = df_cell_details["cell_image"].astype(str).str.strip()

# Ensure paths are absolute (i.e. contain the full path and not just a bit of it)
df_cell_details["cell_tif_path"] = df_cell_details["cell_tif_path"].apply(lambda x: os.path.join(folder_dir, x))
df_cell_details["mask_path"]     = df_cell_details["mask_path"].apply(lambda x: os.path.join(folder_dir, x))

# Load images and masks in CSV order
Ims      = [tifffile.imread(p) for p in df_cell_details["cell_tif_path"]]

Masks_raw = [tifffile.imread(p) for p in df_cell_details["mask_path"]]
def crop_to_bbox(mask):
    # Find the nonzero pixels
    ys, xs = np.where(mask > 0)

    if len(xs) == 0:
        # If mask is empty, just return as-is
        return mask
    # Bounding box coords
    ymin, ymax = ys.min(), ys.max()
    xmin, xmax = xs.min(), xs.max()
    # Crop
    return mask[ymin:ymax+1, xmin:xmax+1]
Masks = [crop_to_bbox(m) for m in Masks_raw]

# Extract FA-planes
Faplane_list = df_cell_details["Faplane"].fillna(1).astype(int).tolist()
Names = df_cell_details["cell_image"]

FAplane_Ims = [
    img[fplane - 1, ...]    # subtract 1 for ImageJ indexing
    for img, fplane in zip(Ims, Faplane_list)
]


### Segmentation

In [ ]:
import numpy as np
import pandas as pd
import os
import napari
import pyclesperanto_prototype as cle
import napari_segment_blobs_and_things_with_membranes as nsbatwm  
import napari_simpleitk_image_processing as nsitk
from skimage import measure, morphology, segmentation
from scipy import ndimage as ndi
from skimage.filters import frangi


for FA_sub, CellMask, Name in zip(FAplane_Ims, Masks, Names):
    print(f"Processing {Name}")
    ### MAPPER segmentation and analysis ###
    MAPPER_channel = FA_sub[0, :, :] 
    Sigma = 1
    MAPPERGaus = cle.gaussian_blur(MAPPER_channel, None, Sigma, Sigma, 0.0)
    Radius = 100
    MAPPERGausSubBG = nsbatwm.subtract_background(MAPPERGaus, Radius)
    MAPPERGausSubBG[MAPPERGausSubBG < 0] = 0
    # laplace box
    MAPPER_lbp = cle.laplace_box(MAPPERGausSubBG)
    # Threshold huang
    MAPPERThresh = nsitk.threshold_huang(MAPPER_lbp)
    MAPPERThreshMask = MAPPERThresh*CellMask
    # Remove small objects
    MAPPER_mask_bool = MAPPERThreshMask.astype(bool)
    MAPPER_mask_clean_bool = morphology.remove_small_objects(MAPPER_mask_bool, min_size=5)
    MAPPERThreshMask_clean = MAPPER_mask_clean_bool.astype(MAPPERThreshMask.dtype)
    # Watershed puncta
    MAPPER_distance = ndi.distance_transform_edt(MAPPER_mask_clean_bool)
    MAPPER_distance_smooth = ndi.gaussian_filter(MAPPER_distance, sigma=1)
    MAPPER_local_maxi = morphology.local_maxima(MAPPER_distance_smooth, connectivity=1)
    MAPPER_local_maxi = MAPPER_local_maxi & MAPPER_mask_clean_bool  # ensure maxima only inside mask
    MAPPER_markers = measure.label(MAPPER_local_maxi) 
    MAPPER_labels_ws = segmentation.watershed(-MAPPER_distance_smooth, MAPPER_markers, mask=MAPPER_mask_clean_bool)
    # Measure properties of puncta
    MAPPER_props_table = measure.regionprops_table(MAPPER_labels_ws, intensity_image=MAPPER_channel,
        properties=['label', 'area', 'centroid', 'eccentricity', 'solidity', 'mean_intensity'])
    MAPPER_df = pd.DataFrame(MAPPER_props_table)

    # Save a tiff of the labelled puncta and also a csv file of the properties
    output_dir = os.path.join(folder_dir, "MAPPER_labels")
    os.makedirs(output_dir, exist_ok=True)
    csv_filename = f"{Name}_MAPPER_labels.csv"
    csv_path = os.path.join(output_dir, csv_filename)
    MAPPER_df.to_csv(csv_path, index=False)
    # Note that this tiff is labels, not a binary mask, to be considered if loading into ImageJ and for future use.
    tif_filename = f"{Name}_MAPPER_labels.tif"
    tif_path = os.path.join(output_dir, tif_filename)
    tifffile.imwrite(tif_path, MAPPER_labels_ws.astype(np.uint16))
    
    ### FA segmentation and analysis ###
    Vinculin_channel = FA_sub[1, :, :]
    ## Generate binary image
    # Gaussian blur
    Sigma = 1
    VincGaus = cle.gaussian_blur(Vinculin_channel, None, Sigma, Sigma, 0.0)
    # Background subtraction
    Radius = 20
    VincGausSubBG = nsbatwm.subtract_background(VincGaus, Radius)
    VincGausSubBG[VincGausSubBG < 0] = 0
    VincGausSubBG = (VincGausSubBG - VincGausSubBG.min()) / (VincGausSubBG.max() - VincGausSubBG.min())
    VincGausSubBG[VincGausSubBG < 0.1] = 0
    # Threshold and apply cell mask
    VincThresh = nsitk.threshold_otsu(VincGausSubBG)
    VincThreshMask = VincThresh * CellMask
    ## Remove small objects
    Vinculin_mask_clean = morphology.remove_small_objects(VincThreshMask.astype(bool), min_size=5)
    ## Watershed FAs
    Vinculin_distance = ndi.distance_transform_edt(Vinculin_mask_clean)
    Vinculin_distance_smooth = ndi.gaussian_filter(Vinculin_distance, sigma=1)
    # Generate ridges
    Vinculin_ridge = frangi(VincGausSubBG, sigmas=range(1, 7))
    Vinculin_ridge_thresh = 0.2 * Vinculin_ridge.max()   # tune 0.15–0.3
    Vinculin_ridge_seeds = Vinculin_ridge > Vinculin_ridge_thresh
    Vinculin_ridge_markers = measure.label(Vinculin_ridge_seeds)
    # Generate seeds
    h = 0.8
    Vinculin_h_min = morphology.h_minima(-Vinculin_distance_smooth, h)
    Vinculin_h_markers = measure.label(Vinculin_h_min)
    # Combine ridges and seeds
    Vinculin_combined_markers = Vinculin_ridge_markers.copy()
    Vinculin_offset = Vinculin_combined_markers.max() + 1
    Vinculin_combined_markers[Vinculin_h_markers > 0] = Vinculin_h_markers[Vinculin_h_markers > 0] + Vinculin_offset
    #Watershed
    Vinc_ws = segmentation.watershed(-Vinculin_distance_smooth, Vinculin_combined_markers, mask=Vinculin_mask_clean)
    # Remove small labels from watershed
    Vinc_ws_clean = morphology.remove_small_objects(Vinc_ws,min_size=15,connectivity=1)
    # Measure properties of FAs
    Vinculin_props_table = measure.regionprops_table(Vinc_ws_clean, intensity_image=Vinculin_channel,
        properties=['label', 'area', 'centroid', 'eccentricity', 'solidity', 'mean_intensity'])
    Vinculin_df = pd.DataFrame(Vinculin_props_table)

    # Save a tiff of the labelled puncta and also a csv file of the properties
    output_dir = os.path.join(folder_dir, "FA_labels")
    os.makedirs(output_dir, exist_ok=True)
    csv_filename = f"{Name}_FA_labels.csv"
    csv_path = os.path.join(output_dir, csv_filename)
    Vinculin_df.to_csv(csv_path, index=False)
    # Note that this tiff is labels, not a binary mask, to be considered if loading into ImageJ and for future use.
    tif_filename = f"{Name}_FA_labels.tif"
    tif_path = os.path.join(output_dir, tif_filename)
    tifffile.imwrite(tif_path, Vinc_ws_clean.astype(np.uint16))
    
    print(f"{Name}: {len(MAPPER_df)} MAPPER objects, {len(Vinculin_df)} FA objects")

In [ ]:
### Testing MAPPER segmentation
import numpy as np
import pandas as pd
import os
import napari
import pyclesperanto_prototype as cle
import napari_segment_blobs_and_things_with_membranes as nsbatwm  
import napari_simpleitk_image_processing as nsitk
from skimage import measure, morphology, segmentation
from scipy import ndimage as ndi

FA_sub = FAplane_Ims[1]  # shape (C, Y, X)
CellMask = Masks[1]
Name = Names[1]
print("File:", Name)
print("Original shape:", FA_sub.shape)

MAPPER_channel = FA_sub[0, :, :] 
print("Selected channel shape:", MAPPER_channel.shape)

Sigma = 1
MAPPERGaus = cle.gaussian_blur(MAPPER_channel, None, Sigma, Sigma, 0.0)

Radius = 100
MAPPERGausSubBG = nsbatwm.subtract_background(MAPPERGaus, Radius)
MAPPERGausSubBG[MAPPERGausSubBG < 0] = 0

# laplace box
MAPPER_lbp = cle.laplace_box(MAPPERGausSubBG)

# threshold huang
MAPPERThresh = nsitk.threshold_huang(MAPPER_lbp)
MAPPERThreshMask = MAPPERThresh*CellMask

# Remove small objects
mask_bool = MAPPERThreshMask.astype(bool)
mask_clean_bool = morphology.remove_small_objects(mask_bool, min_size=5)
MAPPERThreshMask_clean = mask_clean_bool.astype(MAPPERThreshMask.dtype)

# Watershed puncta
distance = ndi.distance_transform_edt(mask_clean_bool)
distance_smooth = ndi.gaussian_filter(distance, sigma=1)
local_maxi = morphology.local_maxima(distance_smooth, connectivity=1)
local_maxi = local_maxi & mask_clean_bool  # ensure maxima only inside mask
markers = measure.label(local_maxi) 
labels_ws = segmentation.watershed(
    -distance_smooth,
    markers,
    mask=mask_clean_bool
)

# Measure properties of puncta
props_table = measure.regionprops_table(
    labels_ws,
    intensity_image=MAPPER_channel,
    properties=['label', 'area', 'centroid', 'eccentricity', 'solidity', 'mean_intensity']
)
df = pd.DataFrame(props_table)

# Save a tiff of the labelled puncta and also a csv file of the properties
output_dir = os.path.join(folder_dir, "MAPPER_labels")
os.makedirs(output_dir, exist_ok=True)

csv_filename = f"{Name}_MAPPER_labels.csv"
csv_path = os.path.join(output_dir, csv_filename)
df.to_csv(csv_path, index=False)

# Note that this tiff is labels, not a binary mask, to be considered if loading into ImageJ and for future use.
tif_filename = f"{Name}_MAPPER_labels.tif"
tif_path = os.path.join(output_dir, tif_filename)
tifffile.imwrite(tif_path, labels_ws.astype(np.uint16))

In [29]:
### Testing FA segmentation
from skimage import measure, morphology, segmentation
from scipy import ndimage as ndi
from skimage.filters import frangi
import numpy as np
import pyclesperanto_prototype as cle
import napari_segment_blobs_and_things_with_membranes as nsbatwm  
import napari_simpleitk_image_processing as nsitk

FA_sub = FAplane_Ims[0]
CellMask = Masks[0]
Name = Names[0]

Vinculin_channel = FA_sub[1, :, :]

## Generate binary image
# Gaussian blur
Sigma = 1
VincGaus = cle.gaussian_blur(Vinculin_channel, None, Sigma, Sigma, 0.0)
# Background subtraction
Radius = 20
VincGausSubBG = nsbatwm.subtract_background(VincGaus, Radius)
VincGausSubBG[VincGausSubBG < 0] = 0
VincGausSubBG = (VincGausSubBG - VincGausSubBG.min()) / (VincGausSubBG.max() - VincGausSubBG.min())
VincGausSubBG[VincGausSubBG < 0.1] = 0

# Threshold and apply cell mask
VincThresh = nsitk.threshold_otsu(VincGausSubBG)
VincThreshMask = VincThresh * CellMask

## Remove small objects
mask_clean = morphology.remove_small_objects(VincThreshMask.astype(bool), min_size=5)

## Watershed FAs
distance = ndi.distance_transform_edt(mask_clean)
distance_smooth = ndi.gaussian_filter(distance, sigma=1)

# Generate ridges
ridge = frangi(VincGausSubBG, sigmas=range(1, 7))

ridge_thresh = 0.2 * ridge.max()   # tune 0.15–0.3
ridge_seeds = ridge > ridge_thresh
ridge_markers = measure.label(ridge_seeds)

# Generate seeds
h = 0.8
h_min = morphology.h_minima(-distance_smooth, h)
h_markers = measure.label(h_min)

# Combine ridges and seeds
combined_markers = ridge_markers.copy()
offset = combined_markers.max() + 1
combined_markers[h_markers > 0] = h_markers[h_markers > 0] + offset
#Watershed
Vinc_ws = segmentation.watershed(-distance_smooth,combined_markers,mask=mask_clean)
# Remove small labels from watershed
Vinc_ws_clean = morphology.remove_small_objects(Vinc_ws,min_size=15,connectivity=1)

# Measure properties of FAs
props_table = measure.regionprops_table(
    Vinc_ws_clean,
    intensity_image=Vinculin_channel,
    properties=['label', 'area', 'centroid', 'eccentricity', 'solidity', 'mean_intensity']
)
df = pd.DataFrame(props_table)

# Save a tiff of the labelled puncta and also a csv file of the properties
output_dir = os.path.join(folder_dir, "FA_labels")
os.makedirs(output_dir, exist_ok=True)

csv_filename = f"{Name}_FA_labels.csv"
csv_path = os.path.join(output_dir, csv_filename)
df.to_csv(csv_path, index=False)

# Note that this tiff is labels, not a binary mask, to be considered if loading into ImageJ and for future use.
tif_filename = f"{Name}_FA_labels.tif"
tif_path = os.path.join(output_dir, tif_filename)
tifffile.imwrite(tif_path, Vinc_ws_clean.astype(np.uint16))

viewer = napari.Viewer() 
viewer.add_image(Vinculin_channel, name="Original Vinculin") 
viewer.add_image(VincGausSubBG, name="background subtracted") 
viewer.add_labels(VincThreshMask, name='Result of Threshold') 
viewer.add_labels(Vinc_ws_clean, name="Watershed") 
napari.run()